In [2]:
from math import inf
from time import time
from copy import deepcopy

In [15]:
class Cores:
    FUNDO_BRANCO  = '\033[1;107m'
    FUNDO_PRETO  = '\033[1;40m'
    FUNDO_AZUL   = '\u001b[44;1m'
    FUNDO_ROSA   = '\u001b[45m'
    AZUL         = '\u001b[36m'
    PRETO        = '\u001b[30;1m'
    BRANCO       = '\u001b[37;1m'
    TRANSPARENTE = '\u001b[47;1m'
    FIM          = '\033[0m'

class Posicao(object):

    def __init__(self, tabuleiro, pasBranca=True):
        self._tabuleiro        = tabuleiro
        self._proxJogadas = None
        self._fimJogo      = False
        self._pasBranca    = pasBranca
        self._avaliacao        = 0

    def __gt__(self, other):
        return self._avaliacao > other.get_avaliacao()

    def __ge__(self, other):
        return self._avaliacao >= other.get_avaliacao()

    def __le__(self, other):
        return self._avaliacao <= other.get_avaliacao()

    def __lt__(self, other):
        return self._avaliacao < other.get_avaliacao()

    def __eq__(self, other):
        return self._avaliacao == other.get_avaliacao()

    def get_tabuleiro(self):
        return self._tabuleiro

    def get_fimJogo(self):
        return self._fimJogo

    def get_pasBranca(self):
        return self._pasBranca

    def set_pasBranca(self, valor):
        self._pasBranca = valor

    def get_avaliacao(self):
        return self._avaliacao

    def set_avaliacao(self, nova_avaliacao):
        self._avaliacao = nova_avaliacao

    def get_proxJogadas(self, forcadas=False):
        if self._proxJogadas is None:
            self.ProxJogadas(forcadas)
        return self._proxJogadas

    def TotalPecas(self):
        numBrancas = 0
        numPretas  = 0

        for i in range(len(self._tabuleiro)):
            for j in range(len(self._tabuleiro[i])):
                if self._tabuleiro[i][j] == 'b':
                    numBrancas += 1
                if self._tabuleiro[i][j] == 'B':
                    numBrancas += 1
                if self._tabuleiro[i][j] == 'p':
                    numPretas += 1
                if self._tabuleiro[i][j] == 'P':
                    numPretas += 1

        return numBrancas, numPretas

    def EncontraJogadaRealizada(self, anterior):
        jogada = []

        for i in range(len(self._tabuleiro)):
            for j in range(len(self._tabuleiro[i])):
                if self._tabuleiro[i][j] != anterior[i][j]:
                    jogada.append((i, j))

        return jogada

    def AvaliarEstadoFinal(self):
        valorBrancas = 0
        valorPretas  = 0

        for i in range(len(self._tabuleiro)):
            for j in range(len(self._tabuleiro[i])):
                if self._tabuleiro[i][j] == 'b':
                    valorBrancas += 2
                if self._tabuleiro[i][j] == 'B':
                    valorBrancas += 3
                if self._tabuleiro[i][j] == 'p':
                    valorPretas += 2
                if self._tabuleiro[i][j] == 'P':
                    valorPretas += 3

        self._avaliacao = valorPretas - valorBrancas

        return self._avaliacao

    def EncontraCapturas(self):
        pecas_capturadas = []

        for i in range(len(self._tabuleiro)):
            for j in range(len(self._tabuleiro[i])):
                if self._pasBranca and (self._tabuleiro[i][j] == 'b' or self._tabuleiro[i][j] == 'B'):
                    jogada = self.JogadaValidaPeca((i, j))
                    for disponivel in jogada:
                        if i - disponivel[0] == 2 or i - disponivel[0] == -2:
                            pecas_capturadas.append((i, j))
                            break
                if not self._pasBranca and (self._tabuleiro[i][j] == 'p' or self._tabuleiro[i][j] == 'P'):
                    jogada = self.JogadaValidaPeca((i, j))
                    for disponivel in jogada:
                        if i - disponivel[0] == 2 or i - disponivel[0] == -2:
                            pecas_capturadas.append((i, j))
                            break
        return pecas_capturadas

    def AvaliarEstado(self):
        valorBrancas = 0
        valorPretas  = 0
        numBrancas   = 0
        numPretas    = 0

        for i in range(len(self._tabuleiro)):
            for j in range(len(self._tabuleiro[i])):
                if self._tabuleiro[i][j] == 'b':
                    numBrancas += 1
                    if 2 < i < 5 and 1 < j < 6:
                        valorBrancas += 50
                    elif i < 4:
                        valorBrancas += 45
                    else:
                        valorBrancas += 40

                if self._tabuleiro[i][j] == 'B':
                    numBrancas += 1
                    valorBrancas += 60

                if self._tabuleiro[i][j] == 'p':
                    numPretas += 1
                    if 2 < i < 5 and 1 < j < 6:
                        valorPretas += 50
                    elif i > 3:
                        valorPretas += 45
                    else:
                        valorPretas += 40

                if self._tabuleiro[i][j] == 'P':
                    numPretas += 1
                    valorPretas += 60

        self._avaliacao = valorPretas - valorBrancas

        if numBrancas == 0:
            self._avaliacao = inf
            self._fimJogo = True
        if numPretas == 0:
            self._avaliacao = -inf
            self._fimJogo = True

        return self._avaliacao

    def ProxJogadas(self, forcadas=False):
        self._proxJogadas = []
        capturas               = []
        todas_jogadas          = []

        for i in range(len(self._tabuleiro)):
            for j in range(len(self._tabuleiro[i])):
                if self._pasBranca:
                    if self._tabuleiro[i][j] == 'b' or self._tabuleiro[i][j] == 'B':
                        vdJogada = self.JogadaValidaPeca((i, j), forcadas)

                        for jogada in vdJogada:
                            if jogada[0] - i == 2 or jogada[0] - i == -2:
                                novo_tabuleiro = self.GeraNovoEstado((i, j), jogada)
                                posicao = Posicao(novo_tabuleiro, not self._pasBranca)
                                capturas.append(posicao)
                            else:
                                novo_tabuleiro = self.GeraNovoEstado((i, j), jogada)
                                posicao = Posicao(novo_tabuleiro, not self._pasBranca)
                                todas_jogadas.append(posicao)

                else:
                    if self._tabuleiro[i][j] == 'p' or self._tabuleiro == 'P':
                        vdJogada = self.JogadaValidaPeca((i, j), forcadas)

                        for jogada in vdJogada:
                            if jogada[0] - i == 2 or jogada[0] - i == -2:
                                novo_tabuleiro = self.GeraNovoEstado((i, j), jogada)
                                posicao = Posicao(novo_tabuleiro, not self._pasBranca)
                                capturas.append(posicao)
                            else:
                                novo_tabuleiro = self.GeraNovoEstado((i, j), jogada)
                                posicao = Posicao(novo_tabuleiro, not self._pasBranca)
                                todas_jogadas.append(posicao)

        if forcadas and len(capturas) > 0:
            self._proxJogadas = capturas
        else:
            self._proxJogadas = capturas + todas_jogadas

    def GeraNovoEstado(self, casa, jogada):
        copia_tabuleiro = deepcopy(self._tabuleiro)
        tipo_casa = copia_tabuleiro[casa[0]][casa[1]]

        if tipo_casa == 'b' or tipo_casa == 'B':
            if jogada[0] == 0:
                copia_tabuleiro[casa[0]][casa[1]] = 'B'
            if casa[0] - jogada[0] == 2 or casa[0] - jogada[0] == -2:
                linha  = casa[0] + (jogada[0] - casa[0]) // 2
                coluna = casa[1] + (jogada[1] - casa[1]) // 2
                copia_tabuleiro[linha][coluna] = '-'

        if tipo_casa == 'p' or tipo_casa == 'P':
            if jogada[0] == 7:
                copia_tabuleiro[casa[0]][casa[1]] = 'B'
            if casa[0] - jogada[0] == 2 or casa[0] - jogada[0] == -2:
                linha  = casa[0] + (jogada[0] - casa[0]) // 2
                coluna = casa[1] + (jogada[1] - casa[1]) // 2
                copia_tabuleiro[linha][coluna] = '-'

        copia_tabuleiro[casa[0]][casa[1]], copia_tabuleiro[jogada[0]][jogada[1]] = copia_tabuleiro[jogada[0]][jogada[1]], copia_tabuleiro[casa[0]][casa[1]]

        return copia_tabuleiro

    def FazerJogada(self, casa, jogada):
        tabuleiro = self.GeraNovoEstado(casa, jogada)
        posicao = None

        for estado in self.get_proxJogadas():
            if tabuleiro == estado.get_tabuleiro():
                posicao = estado
                break

        return posicao

    def JogadaValidaPeca(self, coord, forcadas=False):
        capturas        = []
        vdJogada = []
        casa            = self._tabuleiro[coord[0]][coord[1]]

        if casa != 'b':
            if 0 <= coord[0] < 7:
                if (coord[1] - 1) >= 0:
                    if self._tabuleiro[coord[0] + 1][coord[1] - 1] == '-':
                        vdJogada.append((coord[0] + 1, coord[1] - 1))
                    elif coord[0] + 2 < 8 and coord[1] - 2 >= 0:
                        if self._tabuleiro[coord[0] + 2][coord[1] - 2] == '-':
                            if casa.lower() != self._tabuleiro[coord[0] + 1][coord[1] - 1].lower():
                                capturas.append((coord[0] + 2, coord[1] - 2))

                if (coord[1] + 1) < 8:
                    if self._tabuleiro[coord[0] + 1][coord[1] + 1] == '-':
                        vdJogada.append((coord[0] + 1, coord[1] + 1))
                    elif coord[0] + 2 < 8 and coord[1] + 2 < 8:
                        if self._tabuleiro[coord[0] + 2][coord[1] + 2] == '-':
                            if casa.lower() != self._tabuleiro[coord[0] + 1][coord[1] + 1].lower():
                                capturas.append((coord[0] + 2, coord[1] + 2))

        if casa != 'p':
            if 0 < coord[0] < 8:
                if (coord[1] - 1) >= 0:
                    if self._tabuleiro[coord[0] - 1][coord[1] - 1] == '-':
                        vdJogada.append((coord[0] - 1, coord[1] - 1))
                    elif coord[0] - 2 >= 0 and coord[1] - 2 >= 0:
                        if self._tabuleiro[coord[0] - 2][coord[1] - 2] == '-':
                            if casa.lower() != self._tabuleiro[coord[0] - 1][coord[1] - 1].lower():
                                capturas.append((coord[0] - 2, coord[1] - 2))

                if (coord[1] + 1) < 8:
                    if self._tabuleiro[coord[0] - 1][coord[1] + 1] == '-':
                        vdJogada.append((coord[0] - 1, coord[1] + 1))
                    elif coord[0] - 2 >= 0 and coord[1] + 2 < 8:
                        if self._tabuleiro[coord[0] - 2][coord[1] + 2] == '-':
                            if casa.lower() != self._tabuleiro[coord[0] - 1][coord[1] + 1].lower():
                                capturas.append((coord[0] - 2, coord[1] + 2))

        if forcadas and len(capturas) != 0:
            return capturas

        return capturas + vdJogada

def EscolhePeca(posicao, pecaDisponivel=None):
    while True:
        if pecaDisponivel:
            print("Só é possível mover as peças marcadas!")
        coord = input("Linha x Coluna: \n quit para sair")
        try:
            if coord.lower() == "quit":
                return None

            coordenada = (int(coord) // 10), (int(coord) % 10)
            campo = posicao.get_tabuleiro()[coordenada[0]][coordenada[1]]

            if pecaDisponivel:
                if coordenada in pecaDisponivel:
                    if posicao.get_pasBranca() and campo.lower() == 'b':
                        proxJogadas = posicao.JogadaValidaPeca(coordenada)
                        if len(proxJogadas) != 0:
                            return coordenada
                        else:
                            print("Peca escolhida nao tem jogadas disponiveis!")
                            continue
                    elif not posicao.get_pasBranca() and campo.lower() == 'p':
                        return coordenada
                    else:
                        print("Selecao invalida! Tente novamente.")
            else:
                if posicao.get_pasBranca() and campo.lower() == 'b':
                    proxJogadas = posicao.JogadaValidaPeca(coordenada)
                    if len(proxJogadas) != 0:
                        return coordenada
                    else:
                        print("Peca escolhida nao tem jogadas disponiveis!")
                        continue
                elif not posicao.get_pasBranca() and campo.lower() == 'p':
                    return coordenada
                else:
                    print("Selecao invalida! Tente novamente.")
        except:
            print("Coordenada invalida! Tente novamente.")


def EscolheCasa(vdJogada):
    while True:
        coord = input("Linha x Coluna: \n quit para sair")
        try:
            if coord.lower() == "quit":
                return None
            coordenada = (int(coord) // 10), (int(coord) % 10)
            if coordenada not in vdJogada:
                print("Jogada inválida! Tente novamente.")
            else:
                return coordenada
        except:
            print("Coordenada inválida! Tente novamente")


def JogadasForcadas():
    while True:
        forcadas = input("Habilitar jogadas forçadas? \n sim/nao: ")
        try:
            if forcadas.lower() == 'sim':
                return True
            if forcadas.lower() == 'nao':
                return False
            print("Escolha invalida! Tente novamente.")
        except:
            print("Entrada invalida! Tente novamente.")


def ExibirTabuleiro(tabuleiro, selecionado=None, vdJogada=None):
    for i in range(len(tabuleiro)):
        if i == 0:
            print("    0    1    2    3    4    5    6    7")
        for j in range(len(tabuleiro[i])):
            if j == 0:
                print(i, end=" |")
            if tabuleiro[i][j] == 'b' or tabuleiro[i][j] == 'B':
                if selecionado and ((i, j) in selecionado or (i, j) == selecionado):
                    print(Cores.FUNDO_AZUL + Cores.PRETO + " " + str(tabuleiro[i][j]) + " " + Cores.FIM, end="  ")
                else:
                    print(Cores.FUNDO_BRANCO + " " + str(tabuleiro[i][j]) + " " + Cores.FIM, end="  ")
            elif tabuleiro[i][j] == 'p' or tabuleiro[i][j] == 'P':
                if selecionado and ((i, j) in selecionado or (i, j) == selecionado):
                    print(Cores.FUNDO_AZUL + Cores.PRETO + " " + str(tabuleiro[i][j]) + " " + Cores.FIM, end="  ")
                else:
                    print(Cores.FUNDO_PRETO+ " " + str(tabuleiro[i][j]) + " " + Cores.FIM, end="  ")
            elif vdJogada and (i, j) in vdJogada:
                print(Cores.TRANSPARENTE + Cores.PRETO + str(i) + " " + str(j) + Cores.FIM, end=" ")
            else:
                if selecionado and ((i, j) in selecionado or (i, j) == selecionado):
                    print(Cores.FUNDO_ROSA + Cores.PRETO + " " + str(tabuleiro[i][j]) + " " + Cores.FIM, end="  ")
                else:
                    print(" " + str(tabuleiro[i][j]) + " ", end="  ")
        print("| " + str(i))
    print("    0    1    2    3    4    5    6    7")

def MinMax(posicao, profundidade, max_jogador):
    if profundidade == 0 or posicao.get_fimJogo():
        return posicao.AvaliarEstado()
    if max_jogador:
        max_avaliacao = -inf
        for filho in posicao.get_proxJogadas():
            avaliacao = MinMax(filho, profundidade - 1, False)
            max_avaliacao = max(max_avaliacao, avaliacao)
        posicao.set_avaliacao(max_avaliacao)
        return max_avaliacao
    else:
        min_avaliacao = inf
        for filho in posicao.get_proxJogadas():
            avaliacao = MinMax(filho, profundidade - 1, True)
            min_avaliacao = min(min_avaliacao, avaliacao)
        posicao.set_avaliacao(min_avaliacao)
        return min_avaliacao


def PodaAlphaBeta(posicao, profundidade, alpha, beta, max_jogador, captura_forcada):
    if profundidade == 0 or posicao.get_fimJogo():
        return posicao.AvaliarEstado()
    if max_jogador:
        max_avaliacao = -inf
        for filho in posicao.get_proxJogadas(captura_forcada):
            avaliacao = PodaAlphaBeta(filho, profundidade - 1, alpha, beta, False, captura_forcada)
            max_avaliacao = max(max_avaliacao, avaliacao)
            alpha = max(alpha, avaliacao)
            if beta <= alpha:
                break
        posicao.set_avaliacao(max_avaliacao)
        return max_avaliacao
    else:
        min_avaliacao = inf
        for filho in posicao.get_proxJogadas(captura_forcada):
            avaliacao = PodaAlphaBeta(filho, profundidade - 1, alpha, beta, True, captura_forcada)
            min_avaliacao = min(min_avaliacao, avaliacao)
            beta = min(beta, avaliacao)
            if beta <= alpha:
                break
        posicao.set_avaliacao(min_avaliacao)
        return min_avaliacao


def AlphaBetaSolucao(posicao, profundidade, alpha, beta, max_jogador, captura_forcada):
    if profundidade == 0 or posicao.get_fimJogo():
        return posicao.AvaliarEstadoFinal()
    if max_jogador:
        max_avaliacao = -inf
        for filho in posicao.get_proxJogadas(captura_forcada):
            avaliacao = AlphaBetaSolucao(filho, profundidade - 1, alpha, beta, False, captura_forcada)
            max_avaliacao = min(max_avaliacao, avaliacao)
            alpha = max(alpha, avaliacao)
            if beta <= alpha:
                break
        posicao.set_avaliacao(max_avaliacao)
        return max_avaliacao
    else:
        min_avaliacao = inf
        for filho in posicao.get_proxJogadas(captura_forcada):
            avaliacao = AlphaBetaSolucao(filho, profundidade - 1, alpha, beta, True, captura_forcada)
            min_avaliacao = min(min_avaliacao, avaliacao)
            beta = min(beta, avaliacao)
            if beta <= alpha:
                break
        posicao.set_avaliacao(min_avaliacao)
        return min_avaliacao


def TamProfundidade(tempo_jogada_anterior, profundidade, captura_forcada, num_jogadas):
    if captura_forcada:
        if tempo_jogada_anterior < 0.5 and num_jogadas <= 6:
            return profundidade + 1
        if profundidade > 6 and (tempo_jogada_anterior > 4 or num_jogadas > 6):
            return profundidade - 1
        return profundidade
    else:
        if tempo_jogada_anterior < 0.5:
            return profundidade + 1
        if tempo_jogada_anterior > 4.5:
            return profundidade - 1
        return profundidade


def FimJogo(posicao, contador_pecas, captura_forcada):
    jogadas = posicao.get_proxJogadas(captura_forcada)
    num_pecas = posicao.TotalPecas()
    if num_pecas[0] == 0:
        print("Pretas venceram!")
        return True
    if num_pecas[1] == 0:
        print("Brancas venceram!")
        return True
    if num_pecas[0] + num_pecas[1] == contador_pecas[0]:
        contador_pecas[1] += 1
        if contador_pecas[1] == 50:
            print("Empate!")
            return True
    else:
        contador_pecas[0] = num_pecas[0] + num_pecas[1]
        contador_pecas[1] = 0
    if not jogadas:
        print("FIM")
        return True
    return False


def main():
    tabuleiro = [['-', 'p', '-', 'p', '-', 'p', '-', 'p'],
                 ['p', '-', 'p', '-', 'p', '-', 'p', '-'],
                 ['-', 'p', '-', 'p', '-', 'p', '-', 'p'],
                 ['-', '-', '-', '-', '-', '-', '-', '-'],
                 ['-', '-', '-', '-', '-', '-', '-', '-'],
                 ['b', '.', 'b', '-', 'b', '-', 'b', '-'],
                 ['-', 'b', '-', 'b', '-', 'b', '-', 'b'],
                 ['b', '-', 'b', '-', 'b', '-', 'b', '-']]
    captura_forcada = JogadasForcadas()

    posicao = Posicao(tabuleiro, True)
    tempo_jogada_anterior = 4.5
    profundidade = 6
    nenhuma_captura = [0, 0]

    while True:
        if FimJogo(posicao, nenhuma_captura, captura_forcada):
            break

        print("Nova profundidade: {} ".format(profundidade))
        pecaDisponivel = posicao.EncontraCapturas()
        if captura_forcada:
            ExibirTabuleiro(posicao.get_tabuleiro(), pecaDisponivel)
            peca = EscolhePeca(posicao, pecaDisponivel)
        else:
            ExibirTabuleiro(posicao.get_tabuleiro())
            peca = EscolhePeca(posicao)

        if not peca:
            break
        vdJogada = posicao.JogadaValidaPeca(peca, captura_forcada)
        ExibirTabuleiro(posicao.get_tabuleiro(), peca, vdJogada)
        nova_posicao = EscolheCasa(vdJogada)
        if not nova_posicao:
            break
        tabuleiro_anterior = deepcopy(posicao.get_tabuleiro())
        posicao = posicao.FazerJogada(peca, nova_posicao)
        diferencas = posicao.EncontraJogadaRealizada(tabuleiro_anterior)
        ExibirTabuleiro(posicao.get_tabuleiro(), diferencas)
        print("Jogada do usuário acima \n\n\n")
        if FimJogo(posicao, nenhuma_captura, captura_forcada):
            break
        num_jogadas = len(posicao.get_proxJogadas())
        profundidade = TamProfundidade(tempo_jogada_anterior, profundidade, captura_forcada, num_jogadas)
        tabuleiro_anterior = deepcopy(posicao.get_tabuleiro())
        print("ESPERE! COMPUTADOR CALCULANDO")
        t1 = time()
        num_pecas = posicao.TotalPecas()
        if num_pecas[0] + num_pecas[1] > 6:
            PodaAlphaBeta(posicao, profundidade, -inf, inf, True, captura_forcada)
            posicao = max(posicao.get_proxJogadas())
        else:
            AlphaBetaSolucao(posicao, 20, -inf, inf, True, captura_forcada)
            posicao = max(posicao.get_proxJogadas())
        t2 = time()
        tempo_jogada_anterior = t2 - t1
        diferencas = posicao.EncontraJogadaRealizada(tabuleiro_anterior)
        print(tempo_jogada_anterior)
        ExibirTabuleiro(posicao.get_tabuleiro(), diferencas)
        print("Jogada do computador acima.\n\n\n")


if __name__ == '__main__':
    main()

Habilitar jogadas forçadas? 
 sim/nao: nao
Nova profundidade: 6 
    0    1    2    3    4    5    6    7
0 | -    p    -    p    -    p    -    p   | 0
1 | p    -    p    -    p    -    p    -   | 1
2 | -    p    -    p    -    p    -    p   | 2
3 | -    -    -    -    -    -    -    -   | 3
4 | -    -    -    -    -    -    -    -   | 4
5 | b    .    b    -    b    -    b    -   | 5
6 | -    b    -    b    -    b    -    b   | 6
7 | b    -    b    -    b    -    b    -   | 7
    0    1    2    3    4    5    6    7
Linha x Coluna: 
 quit para sair50
    0    1    2    3    4    5    6    7
0 | -    p    -    p    -    p    -    p   | 0
1 | p    -    p    -    p    -    p    -   | 1
2 | -    p    -    p    -    p    -    p   | 2
3 | -    -    -    -    -    -    -    -   | 3
4 | -   4 1  -    -    -    -    -    -   | 4
5 | b    .    b    -    b    -    b    -   | 5
6 | -    b    -    b    -    b    -    b   | 6
7 | b    -    b    -    b    -    b    -   | 7
    0    1    2    3    4 